# Importing Libraries

In [1]:
#Importing the libraries
import statistics
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

# Data Extraction

In [2]:
company = input("Enter the company name: ")
print(company)

TCS


In [3]:
# Scrape data from web page
url = f'https://www.screener.in/company/{company}/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table')
print(soup.encode("utf-8"))

b'\n<!DOCTYPE html>\n\n<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<title>\n  Tata Consultancy Services Ltd share price | About TCS | Key Insights\n - Screener</title>\n<!-- plausible -->\n<script data-domain="screener.in" defer="" event-user="unregistered" src="https://plausible.screener.in/js/script.pageview-props.tagged-events.js"></script>\n<script>window.plausible = window.plausible || function () { (window.plausible.q = window.plausible.q || []).push(arguments) }</script>\n<!-- FONT -->\n<link href="https://cdn-static.screener.in/inter/inter.43d4904a3137.css" rel="stylesheet"/>\n<link href="https://cdn-static.screener.in/fontello/css/fontello.7a9b496c6953.css" rel="stylesheet"/>\n<!-- CSS -->\n<link href="https://cdn-static.screener.in/css/normalize.a2444750a947.css" media="all" rel="stylesheet"/>\n<link href="https://cdn-static.screener.in/css/skeleton.a559606ed89c.css" media="all" rel="stylesheet"/>\n

In [4]:
search_inputs = soup.find_all(class_='number')

# Initialize an empty list to store text from each element
general_details = []

# Iterate over each element in search_inputs
for search_input in search_inputs:
    # Extract text from the current element and split it into lines
    text = search_input.text.strip()
    lines = text.split('\n')
    # Append non-empty lines to the general_details list
    general_details.extend([line for line in lines if line.strip()])

# Print the general details
print(general_details)


['14,19,412', '3,923', '4,255', '3,156', '32.1', '199', '1.25', '74.8', '60.4', '1.00']


In [5]:
required_tables = {0: "Income Statement", 1: "Profit&Loss",2: "Compounded Sales Growth", 3: "Compounded Profit Growth", 4: "Stock Price CAGR", 5: "ROE", 6: "Balance Sheet", 7: "Cash Flows", 8: "Ratios", 9: "Shareholding_Pattern"}

In [6]:
#Extracting each table data
Quarterly_Results = pd.read_html(str(table[0]))
Quarterly_Results_df = pd.concat(Quarterly_Results, axis=1) # Converting list of dataframes to single dataframe
Quarterly_Results_df.drop(index=11, inplace=True, axis=0)

Profit_Loss = pd.read_html(str(table[1]))
Profit_Loss_df = pd.concat(Profit_Loss, axis=1)

Compounded_Sales_Growth = pd.read_html(str(table[2]))
Compounded_Sales_Growth_df = pd.concat(Compounded_Sales_Growth, axis=1)

Compounded_Profit_Growth = pd.read_html(str(table[3]))
Compounded_Profit_Growth_df = pd.concat(Compounded_Profit_Growth, axis=1)

Stock_Price_CAGR = pd.read_html(str(table[4]))
Stock_Price_CAGR_df = pd.concat(Stock_Price_CAGR, axis=1)

ROE = pd.read_html(str(table[5]))
ROE_df = pd.concat(ROE, axis=1)

Balance_Sheet = pd.read_html(str(table[6]))
Balance_Sheet_df = pd.concat(Balance_Sheet, axis=1)

Cash_Flows = pd.read_html(str(table[7]))
Cash_Flows_df = pd.concat(Cash_Flows, axis=1)

Ratios = pd.read_html(str(table[8]))
Ratios_df = pd.concat(Ratios, axis=1)

Shareholding_Pattern = pd.read_html(str(table[9]))
Shareholding_Pattern_df = pd.concat(Shareholding_Pattern, axis=1)

# Data Processing

- Calculating PE Ratio

In [10]:
#Current PE Ratio
Current_PE_ratio = general_details[4]
print(f"Current PE Ratio: {Current_PE_ratio}")

#Previous Year PE Ratio - FY23
marketcap = general_details[0]
marketcap = marketcap.replace(",", "")
net_profit_23 = Profit_Loss_df.iloc[:,-1][9]
print(marketcap)
print(net_profit_23)
print(Profit_Loss_df)
print(f"FY23 PE Ratio of {company}:", round(int(marketcap)/int(net_profit_23),2))

Current PE Ratio: 74.3
236807
3196
           Unnamed: 0 Dec 2012 Dec 2013 Dec 2014 Dec 2015 Dec 2016 Dec 2017  \
0             Sales +     8335     9101     9855     8175     9141    10010   
1          Expenses +     6509     7153     7816     6620     7292     7913   
2    Operating Profit     1825     1948     2039     1555     1850     2097   
3               OPM %      22%      21%      21%      19%      20%      21%   
4      Other Income +       31       97       87     -391      140      177   
5            Interest       27       37       14        3       91       92   
6        Depreciation      277      330      338      347      354      342   
7   Profit before tax     1553     1678     1774      814     1545     1839   
8               Tax %      31%      33%      33%      31%      35%      33%   
9        Net Profit +     1068     1117     1185      563     1001     1225   
10          EPS in Rs    11.08    11.59    12.29     5.84    10.39    12.71   
11  Dividend Payo

- 5-year Median RoCE

In [20]:
ratios_df = Ratios_df.head(6)
ratios_df

,Unnamed: 0,Dec 2012,Dec 2013,Dec 2014,Dec 2015,Dec 2016,Dec 2017,Dec 2018,Dec 2019,Dec 2020,Dec 2021,Dec 2022,Dec 2023
0,Debtor Days,4,3,4,4,4,3,4,4,5,4,4,4.0
1,Inventory Days,86,79,82,103,107,91,92,107,107,111,110,83.0
2,Days Payable,63,68,71,94,91,99,118,124,115,122,110,92.0
3,Cash Conversion Cycle,28,14,15,12,20,-5,-22,-14,-3,-7,4,-5.0
4,Working Capital Days,-11,-17,-13,-21,-10,-15,-21,-19,-21,-19,-15,-9.0
5,ROCE %,62%,53%,56%,46%,54%,57%,71%,96%,139%,150%,138%,NaN


In [24]:
if not pd.isnull(ratios_df.iloc[:, -1][5]):
    RoCE_5_year_median = statistics.median([int(ratios_df.iloc[:, -1][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -2][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -3][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -4][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -5][5].replace("%", ""))])        
else:
    RoCE_5_year_median = statistics.median([int(ratios_df.iloc[:, -2][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -3][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -4][5].replace("%", "")), 
                                             int(ratios_df.iloc[:, -5][5].replace("%", ""))])
print(f"5-year median RoCE: {RoCE_5_year_median}%")

5-year median RoCE: 138.5%


- Calculating Compounded Sales and Profit Growth

In [33]:
Compounded_Sales_Growth_df.head()

,Compounded Sales Growth,Compounded Sales Growth.1
0,10 Years:,11%
1,5 Years:,10%
2,3 Years:,14%
3,TTM:,7%


In [34]:
print("Compounded Sales Growth:")
csg_ttm = Compounded_Sales_Growth_df['Compounded Sales Growth.1'][3]
csg_3_year = Compounded_Sales_Growth_df['Compounded Sales Growth.1'][2]
csg_5_year = Compounded_Sales_Growth_df['Compounded Sales Growth.1'][1]
csg_10_year = Compounded_Sales_Growth_df['Compounded Sales Growth.1'][0]
print(f"TTM Compounded Sales Growth: {csg_ttm}")
print(f"3 Year Compounded Sales Growth: {csg_3_year}")
print(f"5 Year Compounded Sales Growth: {csg_5_year}")
print(f"10 Year Compounded Sales Growth: {csg_10_year}")

print("\nCompounded Profit Growth:")
cpg_ttm = Compounded_Profit_Growth_df['Compounded Profit Growth.1'][3]
cpg_3_year = Compounded_Profit_Growth_df['Compounded Profit Growth.1'][2]
cpg_5_year = Compounded_Profit_Growth_df['Compounded Profit Growth.1'][1]
cpg_10_year = Compounded_Profit_Growth_df['Compounded Profit Growth.1'][0]
print(f"TTM Compounded Profit Growth: {cpg_ttm}")
print(f"3 Year Compounded Profit Growth: {cpg_3_year}")
print(f"5 Year Compounded Profit Growth: {cpg_5_year}")
print(f"10 Year Compounded Profit Growth: {cpg_10_year}")

Compounded Sales Growth:
TTM Compounded Sales Growth: 7%
3 Year Compounded Sales Growth: 14%
5 Year Compounded Sales Growth: 10%
10 Year Compounded Sales Growth: 11%

Compounded Profit Growth:
TTM Compounded Profit Growth: 11%
3 Year Compounded Profit Growth: 12%
5 Year Compounded Profit Growth: 8%
10 Year Compounded Profit Growth: 9%


# Saving the Data

In [8]:
#Saving the data to Excel
with pd.ExcelWriter(f'..\download\{company}.xlsx') as writer:
    Quarterly_Results_df.to_excel(writer, sheet_name=required_tables[0], index=False)
    Profit_Loss_df.to_excel(writer, sheet_name=required_tables[1], index=False)
    Compounded_Sales_Growth_df.to_excel(writer, sheet_name=required_tables[2], index=False)
    Compounded_Profit_Growth_df.to_excel(writer, sheet_name=required_tables[3], index=False)
    Stock_Price_CAGR_df.to_excel(writer, sheet_name=required_tables[4], index=False)
    ROE_df.to_excel(writer, sheet_name=required_tables[5], index=False)
    Balance_Sheet_df.to_excel(writer, sheet_name=required_tables[6], index=False)
    Cash_Flows_df.to_excel(writer, sheet_name=required_tables[7], index=False)
    Ratios_df.to_excel(writer, sheet_name=required_tables[8], index=False)
    Shareholding_Pattern_df.to_excel(writer, sheet_name=required_tables[9], index=False)